# NIQE and Ma et al. metrics study on satellite imagery

In this notebook we will investigate how the tile size of satellite imagery affects [NIQE](https://ieeexplore.ieee.org/document/6353522) and [Ma et al.](https://www.sciencedirect.com/science/article/pii/S107731421630203X) (short form *Ma*) scores.

We will generate random tile samples of varying size from two satellite sensors, WorldView-2 and GeoEye-1, and use an integration with MATLAB through the [MATLAB Engine API for Python](https://se.mathworks.com/help/matlab/matlab-engine-for-python.html) to evaluate *NIQE* and *Ma et al.* scores

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

from modules.matlab_metrics import *
from modules.image_utils import *
from modules.helpers import *
from modules.tile_generator import *

In [ ]:
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)

SENSORS = ['WV02', 'GE01']
AREAS = ['La_Spezia', 'Toulon']

meta = subset_by_areas_sensor(meta, areas=AREAS, sensors=SENSORS)

# Path to location where individual satellite images are located
DATA_PATH = 'data/toulon-laspezia/' 
DATA_PATH_TILES = 'data/niqe-ma-study'

N_IMAGES = count_images(meta)
N_TILES = 5

SR_FACTOR = 4
MS_SIZES = [32, 48, 64, 72, 96, 120, 128, 144, 160, 168, 192, 224, 256]
PAN_SIZES = [size*SR_FACTOR for size in MS_SIZES]
PAN_AREAS = [size**2 for size in PAN_SIZES]
PAN_N_TILES = [int(N_TILES*PAN_SIZES[-1]/pan_size) for pan_size in PAN_SIZES]
[print(n_tiles) for n_tiles in PAN_N_TILES]

CLOUD_SEA_REMOVAL = False

BATCH_SIZE = 16

In [ ]:
pan_size_dir_paths = []
for i in range(len(MS_SIZES)):
    meta = allocate_tiles(meta, by_partition=False, n_tiles_total=PAN_N_TILES[i])
    ms_size = MS_SIZES[i]
    pan_size = PAN_SIZES[i]
    pan_size_dir_path = str(DATA_PATH_TILES+'/'+str(pan_size))
    pan_size_dir_paths.append(pan_size_dir_path)
    generate_all_tiles(meta, pan_size_dir_path, ms_height_width=[ms_size, ms_size], 
                       sr_factor=SR_FACTOR, save_by_partition=False,
                       cloud_sea_removal=False,
                       print_tile_info=False, save_meta_to_disk=True)

In [ ]:
matlab_engine = MatLabEngine('modules/matlab', ma=True, niqe=True, 
                             output_dtype='uint16', shave_width=4)

In [ ]:
results = pd.DataFrame()
paths_all_tiles = [path.as_posix() for path in list_tiles_in_dir(DATA_PATH_TILES, ms_or_pan='pan')]
results['tile_path'] = paths_all_tiles
results.set_index('tile_path', inplace=True)
results

In [ ]:
for i, pan_size_dir_path in enumerate(pan_size_dir_paths):
    tile_size = PAN_SIZES[i]
    pan_geotiff_tile_paths = list_tiles_in_dir(pan_size_dir_path, ms_or_pan='pan')

    pan_tiles = []
    for pan_geotiff_tile_path in pan_geotiff_tile_paths:
        pan_tile = geotiff_to_ndarray(pan_geotiff_tile_path)
        pan_tiles.append(pan_tile)

    for j in range(len(pan_tiles)):
        niqes_ml, niqes_sk, mas = None, None, None
        batch_range = slice(BATCH_SIZE*j, BATCH_SIZE*(j+1))
        pan_tiles_batch = np.array(pan_tiles[batch_range])
        if pan_tiles_batch.shape[0] == 0:
            break

        tile_paths_batch = pan_geotiff_tile_paths[batch_range]
        
        n_tiles = pan_tiles_batch.shape[0]
        print('Tile size', tile_size, ', shape:', pan_tiles_batch.shape, ', batch:', j, 
              ', batch size:', n_tiles, ', shave width:', matlab_engine.shave_width)

        tic = time.perf_counter()
        niqes_ml = matlab_engine.matlab_niqe_metric(pan_tiles_batch)
        toc = time.perf_counter()
        niqe_ml_sec_per_tile = (toc-tic)/n_tiles
        print('NIQE (matlab) mean:', np.mean(niqes_ml), ', sd:',  np.std(niqes_ml), 
              ', seconds per tile:', niqe_ml_sec_per_tile)
        
        if tile_size > 192:
            tic = time.perf_counter()
            niqes_sk = matlab_engine.skvideo_niqe_metric(pan_tiles_batch)
            toc = time.perf_counter()
            niqe_sk_sec_per_tile = (toc-tic)/n_tiles
            print('NIQE (skvideo) mean:', np.mean(niqes_sk), ', sd:',  np.std(niqes_sk), 
                  ', seconds per tile:', niqe_sk_sec_per_tile)
                            
        tic = time.perf_counter()
        mas = matlab_engine.matlab_ma_metric(pan_tiles_batch)
        toc = time.perf_counter()
        ma_sec_per_tile = (toc-tic)/n_tiles
        print('Ma (matlab) mean:', np.mean(mas), ', sd:',  np.std(mas), 
              ', seconds per tile:', ma_sec_per_tile)
        tile_paths_batch = [path.as_posix() for path in tile_paths_batch]
        for i, tile_path in enumerate(tile_paths_batch):
            try:
                results.loc[tile_path, 'niqe_ml'] = niqes_ml[i]
                results.loc[tile_path, 'niqe_ml_s_per_tile'] = niqe_ml_sec_per_tile
            except TypeError:
                pass
            try:
                results.loc[tile_path, 'niqe_sk'] = niqes_sk[i]
                results.loc[tile_path, 'niqe_sk_s_per_tile'] = niqe_sk_sec_per_tile
            except TypeError:
                pass
            try:
                results.loc[tile_path, 'ma'] = mas[i]
                results.loc[tile_path, 'ma_s_per_tile'] = ma_sec_per_tile
            except TypeError:
                pass

    csv_path = pathlib.Path(DATA_PATH_TILES).joinpath('results-'+str(tile_size)+'.csv')
    results.to_csv(csv_path)
    print('Saved results dataframe as csv at', csv_path.as_posix())        